In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso, LassoCV, Ridge

In [2]:
## EDA --------
# **Read the data dictionary.**
# Determine _what_ missing values mean.
# Figure out what each categorical value represents.
# Identify outliers.
# Consider whether discrete values are better represented as categorical or continuous. (Are relationships to the target linear?)

In [11]:
train = pd.read_csv('C:/Users/james/Documents/GA/Projects/project-2/datasets/train.csv')

In [12]:
train.shape

(2051, 81)

In [13]:
# DROP rows where column is empty
# 'Garage Area'
# 'Garage Cars'
# 'Total Bsmt SF'
train.dropna(subset=['Garage Area'], inplace=True)
train.dropna(subset=['Garage Cars'], inplace=True)
train.dropna(subset=['Total Bsmt SF'], inplace=True)

In [14]:
#function that removes outliers in all columns with a threshold higher than 3
def z_score_col(df, columns):
    for column in columns:
        z_scores = (df[column] - df[column].mean()) / df[column].std()
        outliers = df[np.abs(z_scores) > 3]
        cleaned_df = df.drop(outliers.index)
    return cleaned_df

In [7]:
no_outlier_train = z_score_col(train, xvars)

NameError: name 'xvars' is not defined

In [ ]:
no_outlier_train.head(5)

In [ ]:
no_outlier_train.shape

In [ ]:
train.shape

In [ ]:
# Column Transformer, Scalar, OneHotEncoder
ct = ColumnTransformer([
    ('sc', StandardScaler(), ['X', 'Y']),
    ('poly', PolynomialFeatures(include_bias=False, degree=2), ['FFMC', 'temp', 'wind']),
    ('oh', OneHotEncoder(drop='first', sparse=False), ['month', 'day'])
], remainder='passthrough')

In [ ]:
X_cat = train[categorical_vars]

In [ ]:
ct = ColumnTransformer([
    ('oh', OneHotEncoder(drop='first', sparse=False), ['Fireplace Qu'])
], remainder='passthrough')

In [ ]:
# makes Jupyter show all rows and columns for better visibility
pd.options.display.max_rows = train.shape[1]
pd.options.display.max_columns = train.shape[1]
pd.set_option('display.max_columns', 300)

In [ ]:
# All missing values from highest to lowest
X.isnull().sum().sort_values(ascending=False).head(27)

In [ ]:
# No values in 'Pool QC'
null_columns['Pool QC'].unique()

In [ ]:
# GRAPH: Missing values in each column
train.isnull().sum().plot(kind='barh', figsize=(8,14))
plt.title('Count of Missing Values');

In [ ]:
# shows the highest corellating variables for a single variable model // try top 10? and then retest with more or less variables // dont use both Garage area & Garage Cars??
plt.figure(figsize=(8,10))
sns.heatmap(train.corr(numeric_only=True)[['SalePrice']].sort_values(by='SalePrice', ascending = False), annot=True);

In [15]:
# removed 'Wood Deck SF' and outliers based on Z-score of threshhold: 3
xvars = ['Overall Qual', 'Gr Liv Area', 'Garage Area', 'Garage Cars', 'Total Bsmt SF', 'Year Built', 'Year Remod/Add', 'Full Bath', 'Fireplaces',]
X = train[xvars]
y = train['SalePrice']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

In [20]:
model = LinearRegression()
model.fit(Z_train, y_train)
print(model.score(Z_train, y_train))
print(model.score(Z_test, y_test))

LinearRegression()

In [23]:
# RIDGE Model
ridge_model = Ridge(alpha=10)
ridge_model.fit(Z_train, y_train)
print(ridge_model.score(Z_train, y_train))
print(ridge_model.score(Z_test, y_test))

0.7821179957531557
0.8305356695215075


In [50]:
# Step 5: Refit on all data
Z = sc.fit_transform(X)
ridge_model.fit(Z, y)
print(ridge_model.score(Z, y))

0.7955122735143277


In [51]:
# Refit test.csv
test_scaled = sc.transform(test[['Overall Qual', 'Gr Liv Area', 'Garage Area', 'Garage Cars', 'Total Bsmt SF', 'Year Built', 'Year Remod/Add', 'Full Bath', 'Fireplaces']])

In [52]:
y_pred_submission = ridge_model.predict(test_scaled)

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
y_pred = model.predict(X)

In [ ]:
y_pred.shape

In [ ]:
# R2
metrics.r2_score(y, y_pred)

In [ ]:
# MSE: - Need to fix outliers
mse = metrics.mean_squared_error(y, y_pred)
mse

In [ ]:
# RMSE:
np.sqrt(mse)

In [ ]:
# LINEM ASSUMPTIONS

In [ ]:
# L X and y must be the same size?????
plt.scatter(X, y);

In [ ]:
# I

In [ ]:
# N
resids = y - y_pred
plt.hist(resids, bins=50);

In [ ]:
# E
plt.scatter(y_pred, resids, s=1)
plt.axhline(0, color="orange");

In [ ]:
plt.boxplot(resids)
plt.title('Residuals Boxplot')
plt.show()

### SUBMISSION # 2

In [42]:
test = pd.read_csv('C:/Users/james/Documents/GA/Projects/project-2/datasets/test.csv')

In [40]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,9.048382e+07
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,8,2006,WD,1.049791e+08
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,9,2006,New,7.725659e+07
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2007,WD,6.888377e+07
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,185,0,NaN,NaN,NaN,0,7,2009,WD,8.414219e+07


In [53]:
test['SalePrice'] = y_pred_submission

In [54]:
james_submission = test[['Id', 'SalePrice']]

In [55]:
james_submission.set_index('Id', inplace=True)

In [56]:
james_submission.to_csv('C:/Users/james/Documents/GA/Projects/project-2/data/james_submission_ridge_scaled.csv')